In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks


from extract_bed import extract_bed

In [92]:
bed_path_projections = '../closest/stopcodonsprojections.bed'
closest_df_projections = extract_bed(bed_path_projections)

bed_path_soma = '../closest/stopcodonssoma.bed'
closest_df_soma = extract_bed(bed_path_soma)

/Users/brendagaribay/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (0,6,13) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/brendagaribay/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (0,6,13) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [93]:
fdf_projections = closest_df_projections.groupby('gene_name').filter(lambda x: len(x) > 500)
unique_gene_names = list(set(fdf['gene_name']))

fdf_soma = closest_df_soma.groupby('gene_name').filter(lambda x: len(x) > 500)


In [94]:
def find_gene_specific_histo_peaks(gdf, gene_name, bins=1000, max_length=10000):
    
    df_hist_y, df_hist_x = np.histogram(abs(gdf['distance']), bins=bins, range=(0, max_length))
    
    peaks, properties = find_peaks(df_hist_y, distance=150, width=[1,100], prominence=1)
    widths = properties['widths']
    
    filtered_peaks = []
    filtered_widths = []
    
    for i, peak in enumerate(peaks):
        if df_hist_y[peak] > 0.001*sum(df_hist_y) and df_hist_y[peak] >= 5:
            filtered_peaks.append(peak)
            filtered_widths.append(widths[i])
        
    
            
    df_hist_y = df_hist_y/sum(df_hist_y)
    
    rows = []
    for i,peak in enumerate(filtered_peaks):
        # TODO: sum under curve for isoform abundance
        width = filtered_widths[i]
        abundance = sum(df_hist_y[peak - int(width) : peak + int(width)])
        rows.append({'gene_name': gene_name, 'isoform_length': peak*(max_length/bins), 'isoform_abundane': abundance, 'isoform_spread': filtered_widths[i]})
        
    
    return pd.DataFrame(rows)


In [95]:
gene_name = 'Mobp'
tdf = fdf_projections[fdf_projections['gene_name'] == gene_name]
find_gene_specific_histo_peaks(tdf, gene_name)

,gene_name,isoform_length,isoform_abundane,isoform_spread
0,Mobp,30.0,0.012229,1.622727
1,Mobp,3300.0,0.013412,2.750000
2,Mobp,7350.0,0.554241,7.397321


In [96]:
tdf = fdf_soma[fdf_soma['gene_name'] == gene_name]
find_gene_specific_histo_peaks(tdf, gene_name)

,gene_name,isoform_length,isoform_abundane,isoform_spread
0,Mobp,20.0,0.054484,1.231579
1,Mobp,3300.0,0.161559,4.089286
2,Mobp,7390.0,0.335225,9.361111


In [53]:
dfs = []
for gene in unique_gene_names:
    tmp_df = fdf[fdf['gene_name'] == gene]
    dfs.append(find_gene_specific_histo_peaks(tmp_df, gene))

peaks_df = pd.concat(dfs)

In [ ]:
# which genes transported sum # genes per read which genes have more reads
# new df, each row one gene for i in, column in dataframe total # of reads from geen

In [ ]:
new_df = []
def sum_of_reads(df):
    for gene in df:
        new_df.append(sum(set(gene))) # how to make it so it looks at specific genes? 
        print(new_df)
sum_of_reads()

In [56]:
peaks_df[peaks_df['gene_name'] == 'Plp1']

,gene_name,isoform_length,isoform_abundance
0,Plp1,320.0,0.023825
1,Plp1,2010.0,0.053445
